In [7]:
import random
import requests

user_agents = [
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36 OPR/26.0.1656.60',
    'Opera/8.0 (Windows NT 5.1; U; en)',
    'Mozilla/5.0 (Windows NT 5.1; U; en; rv:1.8.1) Gecko/20061208 Firefox/2.0.0 Opera 9.50',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; en) Opera 9.50',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0',
    'Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2 ',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.71 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
    'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.16 (KHTML, like Gecko) Chrome/10.0.648.133 Safari/534.16',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.101 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11',
    'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)',
    'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0',
    'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; SE 2.X MetaSr 1.0) ',
]

headers = {'User-Agent' : random.choice(user_agents)}
proxies = {'http' : 'http://' + requests.get('http://127.0.0.1:5555/random').text.strip()}

In [9]:
"""
设置
    1、随机请求头
    2、随机IP
试错机制
    1、出错继续执行下一条链接
    2、并将该出错的链接存入列表中，留后续执行完所有链接后从新执行
    3、出错后返回
        （1）出错时的链接
        （2）出错时的IP代理
        （3）出错时的请求头
        （4）出错时的错误 exception 
"""
import random
import pymongo
from lxml import etree
from parsel import Selector
client = pymongo.MongoClient(host='localhost', port=27017)
db     = client['indianhospital']
col    = db['hospital']
error_province_col = db['error_province']
error_hospital_col = db['error_hospital']
"""
一、获取 37 个省份的链接
返回一个包含 37 个省份链接的列表
"""
first_layer_url = 'https://www.medindia.net/patients/hospital_search/hospital_list.asp?utm_source=topnavigation&utm_medium=desktop&utm_content=&utm_campaign=medindia'
headers = {'User-Agent' : random.choice(user_agents)}
proxies = {'http' : 'http://' + requests.get('http://127.0.0.1:5555/random').text.strip()}
try:
    first_layer_res = requests.get(
        url     = first_layer_url,
        headers = headers,
        proxies = proxies
    )
    html = etree.HTML(first_layer_res.text)
    province_href_list = html.xpath('//ul[@class="list-inline"]/div/div/li/a/@href')
    print(province_href_list)  
except:
    print('【ERROR】')
    print('\t获取省份失败')

['https://www.medindia.net/patients/hospital_search/hospitals-andaman-and-nicobar-islands.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-andhra-pradesh.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-arunachal-pradesh.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-assam.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-bihar.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-chandigarh.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-chhattisgarh.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-dadra-and-nagar-haveli.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-daman-and-diu.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-delhi.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-goa.htm', 'https://www.medindia.net/patients/hospital_search/hospitals-gujarat.htm', 'https://www.medindia.net/patients/hospital

In [114]:
"""
二、遍历上面的 37 个省份的链接（通过 上面的列表），
返回每个省份包含的所有医院链接（暂时不设置翻页）的列表
"""
from email import header


error_province = []
error_hospital = []

for province in province_href_list:
    headers = {'User-Agent' : random.choice(user_agents)}
    proxies = {'http' : 'http://' + requests.get('http://127.0.0.1:5555/random').text.strip()}
    try:
        res = requests.get(
            url = province,
            headers=headers,
            proxies = proxies
        )
        html = etree.HTML(res.text)
        hospital_href_list = html.xpath('//h3[@class="vert-small-margin"]/a/@href')
        print('【SUCCDDE hospital_href_list】')

        """测试 hospital_href_list"""
        for hospital in hospital_href_list:
            headers = {'User-Agent' : random.choice(user_agents)}
            proxies = {'http' : 'http://' + requests.get('http://127.0.0.1:5555/random').text.strip()}
            try:
                res = requests.get(
                    url=hospital,
                    headers=headers,
                    proxies=proxies
                )
                print('【SUCCEED hospital】')
                selector = Selector(text = res.text)
                hospital_name = selector.xpath('//div[@class="mi-bg-1"]/../h2/text()').re_first('Address of (.*)')
                address       = ', '.join(selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p[1]//text()').re('\s*(\w.*\w)\s*,*'))
                director      = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Director")]/../text()').re_first('\s*(\w.*\w)\s*')
                email         = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Email")]/../span/text()').re_first('\s*(\w.*\w)\s*')
                phone         = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Phone")]/../span/text()').re_first('\s*(\w.*\w)\s*')
                mobile        = selector.xpath('//div[@class="mi-bg-1"]/div/div/div[contains(@class, "report-content")]/p/b[contains(text(), "Mobile")]/../span/text()').re_first('\s*(\w.*\s)\s*')
                hospital_info = {
                    'name'    : hospital_name,
                    'address' : address,
                    'director': director,
                    'email'   : email,
                    'phone'   : phone,
                    'mobile'  : mobile,
                    'headers' : headers,
                    'proxies' : proxies,
                    'status_code': res.status_code,
                    'res_text': res.text
                }
                
                col.insert_one(hospital_info)   # 医院信息入库
            except:
                print('【ERROR hospital】')
                print('\t', hospital)
                print('\t', proxies)
                error_hospital.append({
                    'province':province,
                    'hospital':hospital,
                    'headers' :headers,
                    'proxies' :proxies
                })
        error_hospital_col.insert_many(error_hospital)   # 错误医院信息入库
    except:
        print('【ERROR hospital_href_list】')
        error_province.append({
            'province':province,
            'headers' :headers,
            'proxies' :proxies
        })
error_province_col.insert_many(error_province)           # 错误省份入库

【SUCCDDE hospital_href_list】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【ERROR hospital_href_list】
	 https://www.medindia.net/patients/hospital_search/hospitals-andaman-and-nicobar-islands.htm
	 {'http': 'http://181.232.190.130:999', 'https': 'http://127.0.0.1:7890', 'ftp': 'http://127.0.0.1:7890'}
【SUCCDDE hospital_href_list】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【SUCCEED hospital】
【ERROR hospital_href_list】
	 https://www.medindia.net/patients/hospital_search/hospitals-andhra-pradesh.ht